In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
from pandas import DataFrame
import time
import gc
import glob
import pickle
import skimage
import skimage.color, skimage.transform, skimage.feature

import cv2

from IPython.display import Image
from IPython.core.display import HTML

from scipy.sparse import csr_matrix

%matplotlib inline 

import os
print(os.listdir("."))

['.git', '.gitignore', '.ipynb_checkpoints', 'Classification.ipynb', 'data', 'feature_extractor.py', 'feature_extractor_linux.py', 'keras_test.py', 'LICENSE', 'out.csv', 'README.md', 'rescale.py', 'submission.csv', 'test.csv', 'test.json', 'train.json', 'Untitled.ipynb', 'Untitled1.ipynb', 'validation.json']


In [104]:
classes = {}
counter = 0
for i in range(1,20):
    classes[i] = []
    for file in glob.glob('./data/train/'+str(i)+'/*.jpeg'):
        classes[i].append(cv2.resize(cv2.imread(file, 0), (48,48)))
        counter += 1
        print(counter, end='\r')

In [105]:
X = []
y = []
for the_class, img_list in classes.items():
    for img in img_list:
        X.append(img)
        y.append(the_class)

In [106]:
X = np.array(X)

In [107]:
X.shape

(28630, 48, 48)

In [108]:
features_hist = np.array([np.histogram(img,bins=20)[0] for img in X])

In [109]:
features_hog = np.array([skimage.feature.hog(img) for img in X])

C:\Users\Michal\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [110]:
conc_features = np.array([np.concatenate((features_hist[i],features_hog[i])) for i in range(len(X))])

In [111]:
conc_features.shape

(28630, 1316)

In [112]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()


In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(conc_features, y, test_size=0.2, random_state=42)

In [123]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# pca = PCA(n_components=60)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

In [118]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

KeyboardInterrupt: 

In [100]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

KeyboardInterrupt: 

In [121]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer 
from sklearn.model_selection import StratifiedKFold
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

mlp = MLPClassifier(hidden_layer_sizes=(200, 100, 10), max_iter=50, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate='adaptive', learning_rate_init=.1)

mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)

Iteration 1, loss = 1.85750600
Iteration 2, loss = 1.33326258
Iteration 3, loss = 1.15078437
Iteration 4, loss = 1.03476077
Iteration 5, loss = 0.94529240
Iteration 6, loss = 0.86209893
Iteration 7, loss = 0.80951539
Iteration 8, loss = 0.75177377
Iteration 9, loss = 0.71314907
Iteration 10, loss = 0.65431064
Iteration 11, loss = 0.62887311
Iteration 12, loss = 0.59143669
Iteration 13, loss = 0.59298556
Iteration 14, loss = 0.56121177
Iteration 15, loss = 0.54018877
Iteration 16, loss = 0.53832546
Iteration 17, loss = 0.51375548
Iteration 18, loss = 0.49120271
Iteration 19, loss = 0.50596718
Iteration 20, loss = 0.54468442
Iteration 21, loss = 0.48600025
Iteration 22, loss = 0.48537788
Iteration 23, loss = 0.48652086
Iteration 24, loss = 0.48980024
Iteration 25, loss = 0.53295587
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Setting learning rate to 0.020000
Iteration 26, loss = 0.33459386
Iteration 27, loss = 0.18068938
Iteration 28, loss = 0.1378105

C:\Users\Michal\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.5986727209221097